In [6]:
from fractions import Fraction
from warnings import warn
import copy
import pandas as pd

### SimplexTable

In [64]:
class SimplexTable:

    def __init__(self, num_vars: int, constraints: list, objective: str, objective_function: str) -> None:
        """
        Метод создает начальную симплекс-таблицу на основе указанных ограничений и целевой функции.
        :param constraints: список ограничений (список строк, например ['1x_1 + 2x_2 >= 4', '2x_3 + 3x_1 <= 5', 'x_3 + 3x_2 = 6'])
        :param num_vars: количество переменных
        :param objective: направление оптимизации ('min' или 'max')
        :param objective_function: целевая функция (например, '2x_1 + 4x_3 + 5x_2')
        :return: None
        """
        self._num_s_vars = 0  # количество избыточных переменных
        self._num_r_vars = 0  # количество дополнительных переменных для балансировки неравенств типа "меньше или равно"
        for expression in constraints:
            if '>=' in expression:
                self._num_s_vars += 1
            elif '<=' in expression:
                self._num_s_vars += 1
                self._num_r_vars += 1
            elif '=' in expression:
                self._num_r_vars += 1
        total_vars = num_vars + self._num_s_vars + self._num_r_vars

        self._coeff_matrix = [[Fraction("0/1") for _ in range(total_vars + 1)] for _ in range(len(constraints) + 1)]
        s_index = num_vars
        r_index = num_vars + self._num_s_vars
        self._r_rows = []  # хранит индексы строк с дополнительными переменными
        for i in range(1, len(constraints) + 1):
            constraint = constraints[i - 1].split(' ')

            for j in range(len(constraint)):

                if '_' in constraint[j]:
                    coeff, index = constraint[j].split('_')
                    if constraint[j - 1] == '-':
                        self._coeff_matrix[i][int(index) - 1] = Fraction("-" + coeff[:-1] + "/1")
                    else:
                        self._coeff_matrix[i][int(index) - 1] = Fraction(coeff[:-1] + "/1")

                elif constraint[j] == '<=':
                    self._coeff_matrix[i][s_index] = Fraction("1/1")  # добавить избыточную переменную
                    s_index += 1

                elif constraint[j] == '>=':
                    self._coeff_matrix[i][s_index] = Fraction("-1/1")  # добавить переменную слака
                    self._coeff_matrix[i][r_index] = Fraction("1/1")  # добавить переменную r
                    s_index += 1
                    r_index += 1
                    self._r_rows.append(i)

                elif constraint[j] == '=':
                    self._coeff_matrix[i][r_index] = Fraction("1/1")  # добавить переменную r
                    r_index += 1
                    self._r_rows.append(i)

            self._coeff_matrix[i][-1] = Fraction(constraint[-1] + "/1")
        self._update_objective_function(objective_function, objective)

    def _update_objective_function(self, objective_function: str, objective: str) -> None:
        """
        Метод добавляет строку с целевой функцией в начальную симплекс-таблицу.
        Строка симплекс-таблицы с целевой функцией формируется из строки objective_function и добавляется в начало таблицы.
        Если целевая функция должна быть максимизирована (параметр objective='max'),
        то строка с целевой функцией симплекса умножается на -1.
        :param objective_function: целевая функция (например, '2x_1 + 4x_3 + 5x_2')
        :param objective: направление оптимизации ('min' или 'max')
        :return: None
        """
        objective_function_coeffs = objective_function.split()
        for i in range(len(objective_function_coeffs)):
            if '_' in objective_function_coeffs[i]:
                coeff, index = objective_function_coeffs[i].split('_')
                if objective_function_coeffs[i-1] == '-':
                    self.set_item(0, int(index)-1, int(coeff[:-1]))
                else:
                    self.set_item(0, int(index) - 1, -int(coeff[:-1]))
                if 'max' in objective:
                    self._coeff_matrix[0][int(index)-1] *= -1

    def get_matrix_params(self) -> tuple:
        """
        Метод возвращает дополнительные параметры симплекс-таблицы, необходимые для реализации метода симплекса:
        :return:
        num_s_vars (int): количество слабых (избыточных переменных), количество дополнительных переменных
        num_r_vars (int): которые приводят ограничения типа "неравенство" к ограничениям типа "равенство"
        r_rows (list): список индексов строк с дополнительными переменными
        
        """
        return self._r_rows, self._num_s_vars, self._num_r_vars

    def get_table(self) -> pd.DataFrame:
        """
        Метод возвращает текущую симплекс-таблицу.
        :return: coff_matrix(list): симплекс-таблица
        """
        return self._coeff_matrix

    def add_zero_row(self) -> None:
        """
        Метод добавляет строку, состоящую из нулей, в текущую симплекс-таблицу
        :return: None
        """
        zero_row = [0 for _ in self._coeff_matrix[1]]
        self._coeff_matrix.append(zero_row)

    def sum_rows_by_index(self, index_left: int, index_right: int) -> list:
        """
        Метод суммирует две строки текущей симплекс-таблицы по их индексам.
        Индексы суммируемых строк задаются в параметрах
        :param index_left: индекс первой строки слагаемого
        :param index_right: индекс второй строки слагаемого
        :return: sum_rows (list): результат суммирования
        """
        row_left = self._coeff_matrix[index_left]
        row_right = self._coeff_matrix[index_right]
        size = len(self._coeff_matrix[index_left])
        sum_rows = [
            row_left[i] + row_right[i] for i in range(size)
        ]
        return sum_rows

    def sum_rows(self, row_left: list, row_right: list) -> list:
        """
        Метод суммирует две строки текущей симплекс-таблицы. Строки задаются в параметрах
        :param row_left: первая строка слагаемое
        :param row_right: вторая строка слагаемое
        :return: sum_rows (list): результат суммирования
        """
        size = len(row_left)
        sum_rows = [row_left[i] + row_right[i] for i in range(size)]
        return sum_rows

    def multiply_const_row(self, const: float, index: int) -> list:
        """
        Метод умножает строку на константу.
        Константа и индекс умножаемой строки указываются в параметрах
        :param const: константа, на которую умножается строка
        :param index: индекс строки, которую нужно умножить на константу
        :return: result (list): результат умножения строки на константу
        """
        result = [const * i for i in self._coeff_matrix[index + 1]]
        return result

    def set_item(self, row, col, value_numerator, value_denominator=1) -> None:
        """
        Метод устанавливает значение элемента в симплекс-таблице по его индексам.
        :param row: индекс строки
        :param col: индекс столбца
        :param value_numerator: числитель значения
        :param value_denominator: знаменатель значения
        :return: None
        """
        self._coeff_matrix[row][col] = Fraction(value_numerator, value_denominator)

    def __setitem__(self, index, data) -> None:
        """ 
        Метод устанавливает значение элемента в симплекс-таблице по его индексу.
        :param index: индекс элемента
        :param data: значение элемента
        :return: None
        """
        self._coeff_matrix[index] = data

    def __getitem__(self, index) -> list:
        """
        Метод возвращает строку симплекс-таблицы по индексу.
        :param index: индекс строки
        :return: row (list): строка симплекс-таблицы
        """
        return self._coeff_matrix[index]

    def __len__(self) -> int:
        """
        Метод возвращает количество строк в симплекс-таблице.
        :return: len (int): количество строк в симплекс-таблице
        """
        return len(self._coeff_matrix)

### SimplexMethod

In [65]:
class SimplexMethod(object):

    def __init__(self, num_vars: int, constraints: list, objective_function: str) -> None:
        """ 
        Метод предназначен для инициализации параметров симплекс-метода:
        - формирование начальной симплекс-таблицы,
        - инициализация базовых переменных,
        - проверка условия разрешимости задачи линейного программирования,
        - инициализация словаря для хранения истории изменений симплекс-таблицы и базовых переменных
        :param num_vars: количество переменных
        :param constraints: список ограничений
         (например, ['1x_1 + 2x_2 >= 4', '2x_3 + 3x_1 <= 5', 'x_3 + 3x_2 = 6'])
        :param objective_function: кортеж, в котором указаны две строковые значения:
         целевая функция (например, '2x_1 + 4x_3 + 5x_2') направление оптимизации ('min' или 'max')
        :return: None
        """
        self.simplex_history = {}
        self.basic_vars_history = {}
        self.num_vars = num_vars
        self.objective = objective_function[0]
        self.objective_function = objective_function[1]
        self.constraints = constraints
        self.simplex_table = SimplexTable(self.num_vars, self.constraints, self.objective, self.objective_function)
        self.r_rows, self.num_s_vars, self.num_r_vars = self.simplex_table.get_matrix_params()
        self.basic_vars = [0 for _ in range(len(self.simplex_table))]
        r_index = self.num_r_vars + self.num_s_vars
        for i in self.basic_vars:
            if i > r_index:
                raise ValueError("Невозможное решение")
        self._delete_r_vars()

    def solve(self) -> tuple:
        """
        Метод предназначен для решения задачи линейного программирования методом симплекса.
        В теле метода каждой итерации цикла проверяется условие оптимальности целевой функции
        относительно указанных ограничений, если это условие верно, задача считается решенной
        и итерации прекращаются, в противном случае выполняется один шаг симплекс-метода
        :return:
        optimum (float) – оптимальное значение целевой функции, полученное методом симплекса
        optimal_plane (dict) - оптимальный план, полученный методом симплекса (решение задачи линейного программирования)
        """
        for row, column in enumerate(self.basic_vars[1:]):
            if self.simplex_table[0][column] != 0:
                const = -self.simplex_table[0][column]
                result = self.simplex_table.multiply_const_row(const, row)
                self.simplex_table[0] = self.simplex_table.sum_rows(self.simplex_table[0], result)
        self.simplex_history['Начальная симплекс-таблица'] = (copy.deepcopy(self.simplex_table.get_table()))
        self.basic_vars_history['Начальная симплекс-таблица'] = copy.deepcopy(self.basic_vars)
        step = 1
        while not self._check_condition():
            key_column = self._find_key_column()
            key_row = self._find_key_row(key_column=key_column)
            self.simplex_step(key_column, key_row)
            self.simplex_history[f'Шаг симплекс-метода {step}'] = copy.deepcopy(self.simplex_table.get_table())
            self.basic_vars_history[f'Шаг симплекс-метода {step}'] = copy.deepcopy(self.basic_vars)

            step += 1
        optimum = self.simplex_table[0][-1]
        optimal_plane = self.get_solution()
        return optimum, optimal_plane

    def get_solution(self) -> dict:
        """
        Метод генерирует оптимальный план на основе текущей симплекс-таблицы.
        :return: optimal_plane (dict) - оптимальный план, полученный методом симплекса (решение задачи линейного программирования)
        """
        optimal_plane = {}
        for i, var in enumerate(self.basic_vars[1:]):
            if var < self.num_vars:
                optimal_plane['x_' + str(var + 1)] = self.simplex_table[i + 1][-1]
        for i in range(0, self.num_vars):
            if i not in self.basic_vars[1:]:
                optimal_plane['x_' + str(i + 1)] = 0
        return optimal_plane

    def simplex_step(self, key_column: int, key_row: int) -> None:
        """
        Метод выполняет один шаг симплекс-алгоритма:
        - ввод новых базовых переменных и вывод старых переменных из базиса,
        - поиск разрешающего элемента,
        - нормализация разрешающей строки к разрешающему элементу,
        - обнуление разрешающего столбца.
        :param key_column: индекс разрешающего столбца
        :param key_row: индекс разрешающей строки
        :return: None
        """
        self.basic_vars[key_row] = key_column
        pivot = self.simplex_table[key_row][key_column]
        self._normalize_to_pivot(key_row, pivot)
        self._make_key_column_zero(key_column, key_row)

    def _check_condition(self) -> bool:
        """
        Метод проверяет условие оптимальности целевой функции для текущей симплекс-таблицы
         (все значения в строке целевой функции отрицательные или равны нулю)
        :return: condition (bool): True, если условие оптимальности выполнено, False в противном случае
        """
        F = self.simplex_table[0]
        negative_values = list(filter(lambda x: x <= 0, F))
        condition = len(negative_values) == len(F)
        return condition

    def _find_key_column(self) -> int:
        """
        Метод предназначен для поиска разрешающего столбца в текущей симплекс-таблице
        :return: key_column (int):  индекс разрешающего столбца
        """
        key_columns = 0
        for i in range(0, len(self.simplex_table[0]) - 1):
            if abs(self.simplex_table[0][i]) >= abs(self.simplex_table[0][key_columns]):
                key_columns = i

        return key_columns

    def _find_key_row(self, key_column: int) -> int:
        """
        Метод предназначен для поиска разрешающей строки в разрешающем столбце текущей симплекс-таблицы.
        :param key_column: индекс разрешающего столбца
        :return: key_row (int): индекс разрешающей строки
        """
        min_val = float("inf")
        key_row = 0
        for i in range(1, len(self.simplex_table)):
            if self.simplex_table[i][key_column] > 0:
                val = self.simplex_table[i][-1] / self.simplex_table[i][key_column]
                if val < min_val:
                    min_val = val
                    key_row = i
        if min_val == float("inf"):
            raise ValueError("Бесконечное решение")
        if min_val == 0:
            warn("Дегенерация")
        return key_row

    def _normalize_to_pivot(self, key_row: int, pivot: float) -> None:
        """
        Метод делит разрешающую строку текущей симплекс-таблицы на разрешающий элемент
        :param key_row: индекс разрешающей строки
        :param pivot: разрешающий элемент
        :return: None
        """
        for i in range(len(self.simplex_table[0])):
            self.simplex_table[key_row][i] /= pivot

    def _make_key_column_zero(self, key_column: int, key_row: int) -> None:
        """
        Метод обнуляет элементы разрешающего столбца текущей симплекс-таблицы
        за исключением элемента, стоящего в разрешающей строке по методу Жордано-Гаусса
        :param key_column: индекс разрешающего столбца
        :param key_row: индекс разрешающей строки
        :return: None
        """
        num_columns = len(self.simplex_table[0])
        for i in range(len(self.simplex_table)):
            if i != key_row:
                factor = self.simplex_table[i][key_column]
                for j in range(num_columns):
                    self.simplex_table[i][j] -= self.simplex_table[key_row][j] * factor

    def _delete_r_vars(self) -> None:
        """
        Метод удаляет из симплекс-таблицы дополнительные переменные r
        :return: None
        """
        for i in range(len(self.simplex_table)):
            non_r_length = self.num_vars + self.num_s_vars + 1
            length = len(self.simplex_table[i])
            while length != non_r_length:
                del self.simplex_table[i][non_r_length-1]
                length -= 1

    def _phase1(self) -> None:
        # Функция цели здесь - минимизировать r1 + r2 + r3 + ... + rn
        """ 
        Метод предназначен для выполнения фазы 1 симплекс-метода.
        Фаза 1 симплекс-метода предназначена для поиска начального базисного решения задачи линейного программирования.
        В теле метода выполняется инициализация дополнительных переменных r, добавление их в симплекс-таблицу,
        итеративное выполнение шагов симплекс-метода до тех пор, пока не будет найдено начальное базисное решение.
        :return: None
        """
        r_index = self.num_vars + self.num_s_vars
        for i in range(r_index, len(self.simplex_table[0]) - 1):
            self.simplex_table.set_item(0, i, -1)
        for i in self.r_rows:
            self.simplex_table[0] = self.simplex_table.sum_rows_by_index(0, i)
            self.basic_vars[i] = r_index
            r_index += 1
        s_index = self.num_vars
        for i in range(1, len(self.basic_vars)):
            if self.basic_vars[i] == 0:
                self.basic_vars[i] = s_index
                s_index += 1
        step = 0
        while not self._check_condition():
            key_column = self._find_key_column()
            key_row = self._find_key_row(key_column=key_column)
            self.simplex_step(key_column, key_row)
            self.simplex_history[f'Шаг симплекс-метода {step}'] = copy.deepcopy(self.simplex_table.get_table())
            self.basic_vars_history[f'Шаг симплекс-метода {step}'] = copy.deepcopy(self.simplex_table.get_table())
            step += 1


### GomoryMethod

In [66]:
class GomoryMethod(SimplexMethod):

    def __init__(self, num_vars: int, constraints: list, objective_function: tuple) -> None:
        """
        Метод вызывает конструктор родительского класса SimplexMethod и инициализирует параметры симплекс-метода.
        В методе формируется решение симплекс-метода для заданных ограничений и целевой функции.
        :param num_vars: количество переменных
        :param constraints: список ограничений
         (например, ['1x_1 + 2x_2 >= 4', '2x_3 + 3x_1 <= 5', 'x_3 + 3x_2 = 6'])
        :param objective_function: кортеж, в котором указаны два строковых значения:
         целевая функция (например, '2x_1 + 4x_3 + 5x_2') направление оптимизации ('min' или 'max')
        :return: None
        """
        super().__init__(num_vars, constraints, objective_function)
        self.clipping_history = []
        self.simplex_vals, self.simplex_solution = self.solve()

    def integer_solve(self) -> tuple:
        """
        Метод предназначен для решения задачи целочисленного линейного программирования по заданным ограничениям и целевой функции.
        Изначально проверяется условие целочисленности полученного решения методом симплекса,
        если решение целочисленное – оно выводится в ответ.
        На каждой итерации цикла проверяется условие целочисленной оптимальности целевой функции.
        Если это условие выполняется, задача считается решенной и прекращаются итерации,
        возвращается полученное целочисленное оптимальное значение функции и целочисленный оптимальный план.
        В противном случае выполняется один шаг алгоритма метода отсечения плоскостей (метод Гомори):
        - формирование отсечения и добавление его в текущую симплекс-таблицу
        - выполнение одного шага метода симплекса
        :return:
        integer_optimum (float): целочисленное оптимальное значение целевой функции, полученное методом Гомори
        integer_optimal_plane (list): целочисленный оптимальный план, полученный методом Гомори
        """
        if self._check_integer_condition():
            return self.simplex_table[0][-1], self.simplex_solution
        self.simplex_history[f'Начальный метод Гомори'] = copy.deepcopy(self.simplex_table.get_table())
        self.basic_vars_history[f'Начальный метод Гомори'] = copy.deepcopy(self.basic_vars)
        step = 1
        while not self._check_integer_condition():
            self._add_clipping()
            key_row = self._get_key_row()
            key_column = self._get_key_column(key_row)
            self.simplex_step(key_column, key_row)
            self.simplex_history[f'Шаг метода Гомори {step}'] = copy.deepcopy(self.simplex_table.get_table())
            self.basic_vars_history[f'Шаг метода Гомори {step}'] = copy.deepcopy(self.simplex_table.get_table())
            step += 1
        integer_optimum = self.simplex_table[0][-1]
        integer_optimal_plane = self.get_solution()
        return integer_optimum, integer_optimal_plane

    def _check_integer_condition(self) -> bool:
        """
        Метод проверяет условие целочисленной оптимальности целевой функции
        (все значения в строке целевой функции текущей симплекс-таблицы отрицательные или равны 0 и являются целыми числами)
        :return: condition (bool): True или False, в зависимости от выполнения условия целочисленной оптимальности
        """
        B = [x[-1] for x in self.simplex_table[1:]]
        integer_values = list(filter(lambda x: x.denominator == 1, B))
        positive_values = list(filter(lambda x: x >= 0, B))
        condition1 = len(positive_values) == len(B)
        condition2 = len(integer_values) == len(B)
        condition = condition1 and condition2
        return condition

    def _find_max_fractional_index(self) -> int:
        """
        Метод ищет индекс строки текущей симплекс-таблицы, в которой находится
        максимальное значение дробной части элемента
        :return: max_fractional_index (int): индекс строки, содержащей элемент с максимальной дробной частью
        """
        max_fractional_part_i = 1
        for i in range(1, len(self.simplex_table)):
            curr_fractional_part = abs(float(self.simplex_table[i][-1])) % 1
            max_fractional_part = abs(float(self.simplex_table[max_fractional_part_i][-1])) % 1
            if curr_fractional_part > max_fractional_part:
                max_fractional_part_i = i
        return max_fractional_part_i

    def _add_clipping(self) -> None:
        """
        Метод формирует новое отсечение метода Гомори:
        - поиск строки, содержащей элемент с максимальной дробной частью,
        - составление неравенства отсечения Гомори на основе найденной строки,
        - приведение полученного неравенства к равенству путем добавления новой переменной,
        - введение новой переменной в базис и добавление отсечения в текущую симплекс-таблицу
        :return: None
        """
        index = self._find_max_fractional_index()
        self.simplex_table.add_zero_row()
        for i in range(len(self.simplex_table)):
            if i == len(self.simplex_table)-1:
                self.simplex_table[i].insert(-1, 1)
            else:
                self.simplex_table[i].insert(-1, 0)
        clipping_coeffs = []
        for j, coeff in enumerate(self.simplex_table[index]):
            if coeff != 0:
                self.simplex_table.set_item(-1, j, -(coeff.numerator % coeff.denominator), coeff.denominator)
                clipping_coeffs.append(self.simplex_table[-1][j])
        self.clipping_history.append(clipping_coeffs[1:])
        self.basic_vars.append(len(self.simplex_table) - 1)

    def _get_key_row(self) -> int:
        """
        Метод находит разрешающую строку в текущей симплекс-таблице
        :return: key_row (int): индекс разрешающей строки
        """
        beta = [x[-1] if x[-1] < 0 else 0 for x in self.simplex_table[1:]]
        key_row = 0
        for b in range(len(beta)):
            if abs(beta[b]) >= abs(beta[key_row]):
                key_row = b
        return key_row + 1

    def _get_key_column(self, key_row):
        """
        Метод находит разрешающий столбец в текущей симплекс-таблице
        :param key_row: индекс разрешающей строки
        :return: key_column (int): индекс разрешающего столбца
        """
        tetha = [x / y if y < 0 else float('inf') for x, y in
                 zip(self.simplex_table[0][:-2], self.simplex_table[key_row][:-2])]
        key_column = 0
        for t in range(len(tetha)):
            if tetha[t] <= tetha[key_column]:
                key_column = t
        return key_column

### Пример

In [68]:
# Задаём целевую функцию и ограничения
objective_function = ('maximize', '4x_1 + 5x_2 + 7x_3')
constraints = ['7x_1 + 4x_2 + 8x_3 <= 51', '1x_1 + 4x_2 + 9x_3 <= 38', '0x_1 + 7x_2 + 3x_3 <= 29']

# Создаём объект метода Гомори с указанными переменными и ограничениями
gomory = GomoryMethod(num_vars=3, constraints=constraints, objective_function=objective_function)

# Выполняем целочисленное решение
gomory.integer_solve()

# Получаем историю таблиц симплекс-метода и историю базисных переменных
history = gomory.simplex_history
basic_vars_history = gomory.basic_vars_history

# Выводим шаги метода Гомори
print('Шаги метода Гомори:')
for step, table in history.items():
    print(step)
    # Создаём таблицу симплекс-метода с учетом истории базисных переменных
    simplex_table = pd.DataFrame(
        table,
        columns=[f'x{i+1}' if i < len(table[0])-1 else 'b' for i in range(len(table[0]))],
        index=[f'x{i+1}' if i != 0 else 'f(x)' for i in range(len(basic_vars_history[step]))]  # Обновляем индексы
    )
    print(simplex_table)
    print()

# Выводим оптимальную точку
print('Оптимальная точка:')
solution = gomory.get_solution()
for var, value in solution.items():
    print(var, value)


Шаги метода Гомори:
Начальная симплекс-таблица
     x1  x2   x3  x4  x5 x6    b
f(x)  0  85  191  -4  24  0  708
x2    7   4    8   1   0  0   51
x3    1   4    9   0   1  0   38
x4    0   7    3   0   0  1   29

Шаг симплекс-метода 1
          x1    x2 x3  x4    x5 x6       b
f(x)  -191/9   1/9  0  -4  25/9  0  -886/9
x2      55/9   4/9  0   1  -8/9  0   155/9
x3       1/9   4/9  1   0   1/9  0    38/9
x4      -1/3  17/3  0   0  -1/3  1    49/3

Шаг симплекс-метода 2
     x1      x2 x3      x4      x5 x6        b
f(x)  0   91/55  0  -29/55  -17/55  0  -425/11
x2    1    4/55  0    9/55   -8/55  0    31/11
x3    0   24/55  1   -1/55    7/55  0    43/11
x4    0  313/55  0    3/55  -21/55  1   190/11

Шаг симплекс-метода 3
     x1 x2 x3        x4       x5       x6           b
f(x)  0  0  0  -170/313  -62/313  -91/313  -13665/313
x2    1  0  0    51/313  -44/313   -4/313     813/313
x3    0  0  1    -7/313   49/313  -24/313     809/313
x4    0  1  0     3/313  -21/313   55/313     950/313